<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/BlockChain/python_blockchain_training_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile hash_util.py
import hashlib as hl
import json

def hash_string_256(string):
  return hl.sha256(string).hexdigest()

def hash_block(block):
  return hash_string_256(json.dumps(block, sort_keys=True).encode())


Writing hash_util.py


In [0]:
#  %writefile blockchain.py
# pickle
#####################################################
import functools
import hashlib as hl
import json
import pickle
from collections import OrderedDict

from hash_util import hash_string_256, hash_block

#global variable
MINING_REWARD = 10

genesis_block = {
          'previous_hash': '', 
          'index':0 , 
          'transactions':[],
          'proof':100
  }
blockchain = [genesis_block]
open_transactions = []
owner = 'Sumith'
participants = {owner}

def load_data():
  with open('blockchain.p', mode='rb') as f:
    file_content = pickle.loads(f.read())
    #print(file_content)
    global blockchain
    global open_transactions
    blockchain = file_content['chain']
    open_transactions = file_content['ot']
 
load_data()

def save_data():
  with open('blockchain.p', mode='wb') as f:
    save_data = {
        'chain': blockchain,
        'ot': open_transactions
    }
    f.write(pickle.dumps(save_data))

def get_last_blockchain_value():
  """ returns the last value of the current bloackchain """
  #function to return the value
  if len(blockchain) < 1 :
    return None
  return blockchain[-1]

def verify_transaction(transaction):
  sender_balance = get_balance(transaction['sender'])
  return sender_balance >= transaction['amount']


#adding kwargs to function and setting up default value
#transaction_amount is local varible
def add_transaction(recipient,sender=owner, amount=1.0):
  """ Appends a new value as well as the last blockchain value to teh block

  Arguments:
    : Sender :  sender of coins
    : recipent : recipient of coin
  """
  #appending values to blockchian and retaining the last value
  #transaction = {'sender': sender,
  #               'recipient': recipient, 
  #               'amount' : amount }
  transaction = OrderedDict([('sender', sender),
                            ('recipient', recipient), 
                            ('amount' , amount)])
  if verify_transaction(transaction):
    open_transactions.append(transaction)
    participants.add(sender)
    participants.add(recipient)
    save_data()
    return True
  return False


def valid_proof(transactions, last_hash, proof):
  guess = (str(transactions) + str(last_hash) + str(proof)).encode()
  guess_hash = hash_string_256(guess)
  print(guess_hash)
  return guess_hash[0:2] == '00'

def proof_of_work():
  last_block = blockchain[-1]
  last_hash = hash_block(last_block)
  proof = 0
  while not valid_proof(open_transactions, last_hash, proof):
    proof += 1
  return proof
  

def get_balance(participant):
  tx_sender = [[tx['amount'] for tx in block['transactions'] if tx['sender'] == participant] for block in blockchain]
  open_tx_sender = [tx['amount'] for tx in open_transactions if tx['sender'] == participant]
  tx_sender.append(open_tx_sender)
  amount_sent = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_sender,0)
  tx_recipient = [[tx['amount'] for tx in block['transactions'] if tx['recipient'] == participant] for block in blockchain]
  amount_received = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_recipient,0)
  return  amount_received - amount_sent

def mine_block():
  last_block = blockchain[-1]
  #list comprehensions
  hashed_block = hash_block(last_block)
  proof = proof_of_work() 
  #reward_transaction = {
  #    'sender': 'MINING',
  #    'recipient': owner,
  #    'amount': MINING_REWARD
  #}
  reward_transaction = OrderedDict(
                      [('sender', 'MINING'),
                      ('recipient', owner),
                      ('amount', MINING_REWARD)])
  copied_transactions = open_transactions[:]
  copied_transactions.append(reward_transaction)
  block = {
          'previous_hash': hashed_block, 
          'index' : len(blockchain), 
          'transactions' : copied_transactions,
           'proof': proof
  }
  blockchain.append(block)
  #save_data()
  return True

#using function for reusable code.
def get_transaction_value():
  """ Retruns the input of the user(a new transaction amount) as a float
  """
  tx_recipient = input('Enter the recipent of the transaction: ') 
  tx_amount = float(input('Your trasaction amount please: ')) 
  return (tx_recipient, tx_amount)
  
def get_user_choice():
  user_input = input('Your Choice : ')
  return user_input

def print_blockchain_elements():
  #use for loop to output the blockchain list console
  for block in blockchain:
    print('outputting block : ', block)
  else:
    print('-' * 20)

def verify_chain():
  """verify current blockchain"""
  for (index, block) in enumerate(blockchain):
    if index == 0:
      continue
    if block['previous_hash'] != hash_block(blockchain[index -1]):
      return False
    if not valid_proof(block['transactions'][:-1], block['previous_hash'], block['proof']):
      print('Prood of work invalid')
      return False
  return True

def verify_transactions():
  return all([verify_transaction(tx) for tx in open_transactions])
#  is_valid = True
#  for tx in open_transactions:
#    if verify_transaction(tx):
#      is_valid = True
#    else:
#      is_valid = False
  
waiting_for_input = True

while waiting_for_input:
  print('Please choose')
  print('1: Add a new transaction value')
  print('2: Mine a new block')
  print('3: Output the blockchain blocks')
  print('4: output participants')
  print('5: check transaction validity')
  print('q : Quit ')
  user_choice = get_user_choice()
  if user_choice == '1':
    tx_data = get_transaction_value()
    recipient, amount = tx_data
    if add_transaction(recipient, amount=amount):
      print('Added transaction')
    else:
      print('Tranaction failed')
    print(open_transactions)
  elif user_choice == '2':
    if mine_block():
      open_transactions = []
      save_data()
  elif user_choice == '3':
    print_blockchain_elements()
  elif user_choice == '4':
    print(participants)
  elif user_choice == '5':
    if verify_transactions():
      print("All Transactions are valid")
    else:
      print('There are invalid transaction')
  elif user_choice == 'h':
    if len(blockchain) >=1:
      blockchain[0] = {
          'previous_hash': '', 
          'index':0 , 
          'transactions':[{'sender':'Chris', 'recipient': 'Max', 'amount': 100}]
      }
  elif user_choice == 'q':
    waiting_for_input = False
  else:
    print('Input was invalid, please pick a value from the list!')    
  
  if not verify_chain():
    print_blockchain_elements()
    print('Invalid block chain!')
    break
  print('Balance of {} : {:6.2f}'.format(owner, get_balance(owner)))  
  print('Choice registered !')
else:
  print('User left')

print('Done')


Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: output participants
5: check transaction validity
q : Quit 
Your Choice : 2
89317ae45637e638f93972d83f22e2f7b246fe8df5035be35d97f17427edcba8
3d40c9473ad211445e8e7cc3202de823ef3bd78fed6200669eadce4ef22ee408
1238aefb4a772bed960178cccb454ee75f504d1524790b446ab035de3869796e
9fc0349a3455a86071196f42f9e337060f19ba8ccc8580757a70ea6f08cca840
350e074fb375f21c224e152be47696c84838568ba8ca0fb3a5d43ac8fb491396
16ee0b6bd161148e7b23c50f6d12e9cca3b4ae74e632973f70dcb0e8fadd8034
cf28bf14fc9b2bfe5e79b4ebab902c8b8748fe65ba2ac38bffd0155e356c250c
109a7d2e311ec76e3374bf767d6f005527af36de2f3eda90e3013f59d61f5491
af30142a9d8e164ecf7ef4f4031b24c345a359b82aaa6810951ce942468c3f5b
aca6f94674c33825e514b799b2103d5eb44d832656c405de61bd911ed3a82f90
f3560d0f3ba0c18a848b4dba58a126e62667f99c8493bedb905a1d91ef727824
4c1809ebb9231a9b80fef43a336a21a697a5668b8fb2c3c698712c3e05193631
424f40210b62770c0063cfc4dc8cdf0b69de492e6181

In [3]:
!cat blockchain.p

UnicodeDecodeError: ignored